In [8]:
from math import log

#### 1、计算给定数据的香农熵


In [9]:
def calcShannonEnt(dataSet):
    numEntries=len(dataSet)  #获得数据集的行数
    labelCounts={}  #字典：key表示类标签，value表示在数据集中每个类标签出现的次数
    for featVec in dataSet:   #以行为单位遍历数据集
        currentLabel=featVec[-1]  #每一行的最后一个数据是该行数据的类标签
        if currentLabel not in labelCounts.keys(): 
            labelCounts[currentLabel]=0
        labelCounts[currentLabel]+=1
    shannonEnt=0.0
    for key in labelCounts:  #遍历字典的key，计算所有类别所有可能值包含的信息期望值
        prob=float(labelCounts[key])/numEntries
        shannonEnt-=prob*log(prob,2)
    return shannonEnt

#### 2、按照给定特征划分数据集
dataSet --  待划分的数据集  
axis -- 划分数据集的特征 （索引）   
value -- 需要返回的特征的值  

In [27]:
def splitDataSet(dataSet,axis,value):
    retDataSet=[]
    for featVec in dataSet:  #遍历数据集的每一行
        if featVec[axis] == value:  #若该行数据在axis索引位置上的值等于value，将该行数据在axis上的值剔除，并保存到列表中返回
            reducedFeatVec=featVec[:axis]
            reducedFeatVec.extend(featVec[axis+1:])
            retDataSet.append(reducedFeatVec)
    return retDataSet
            

#### 3、选择最好的数据集划分方式

以海洋生物的数据为例：  
1,1,'yes'  
1,0,'yes'  
1,0,'no'  
0,1,'no'  
0,1,'no'

In [28]:
def chooseBestFeatureToSplit(dataSet):
    numFeatures=len(dataSet[0])-1  #计算特征的数量，例子中为2，不包含最后一列的类标签
    baseEntropy=calcShannonEnt(dataSet) #计算原始数据集的香农熵
    bestInfoGain=0.0;bestFeature=-1
    for i in range(numFeatures): #遍历特征，逐一选择最优划分数据集的特征
        featList=[example[i] for example in dataSet]  #获得每一列的取值，即每个特征的取值
        uniqueVals=set(featList) #对每个特征的取值去重，获得特征中的所有唯一属性值，例子中分别为：{0,1}和{0,1}
        newEntropy=0.0
        for value in uniqueVals:  #遍历特征的唯一属性值 
            subDataSet=splitDataSet(dataSet,i,value) #以例子中第一个特征为例，第一次返回：[1,no],[1,no];第二次返回[1,yes],[0,yes],[0,no]
            prob=len(subDataSet)/float(len(dataSet))
            newEntropy+=prob*calcShannonEnt(subDataSet) #对所有唯一特征值得到的熵求和
        infoGain=baseEntropy-newEntropy  #计算信息增益
        if(infoGain>bestInfoGain):  #信息增益最大的就是划分数据集最好的特征
            bestInfoGain=infoGain
            bestFeature=i
    return bestFeature  #返回最好特征的索引

In [20]:
import operator

#### 4、多数表决确定叶子节点的分类
该方法用于，若数据集已经处理了所有的属性，但是类标签依然不是唯一的这种情况。

In [21]:
def majorityCnt(classList):
    classCount={}
    for vote in classList:
        if vote not in classCount.keys():
            classCount[vote]=0
        classCount[vote]+=1
    sortedClassCount=sorted(classCount.items(),key=operator.itemgetter(1),reverse=True)
    return sortedClassCount[0][0]

#### 5、创建树
dataSet -- 数据集  
labels -- 标签列表


In [22]:
def createTree(dataSet,labels):
    classList=[example[-1] for example in dataSet]  #获得数据集的所有类标签
    if classList.count(classList[0])==len(classList):  #递归结束的第一个条件：数据集的所有类标签完全相同
        return classList[0]
    if len(dataSet[0])==1: #递归结束的第二个条件：数据集的列数为1，即使用完了所有特征，仍不能将数据集划分为唯一的分组
        return majorityCnt(classList) #采用多数表决法
    bestFeat=chooseBestFeatureToSplit(dataSet)  #获得最好划分数据集的特征索引
    bestFeatLabel=labels[bestFeat] #获得该特征的标签名
    myTree={bestFeatLabel:{}}  #以字典的方式保存构建的决策树
    del(labels[bestFeat])
    featValues=[example[bestFeat] for example in dataSet]  #获得该数据集下该特征的所有取值
    uniqueVals=set(featValues) #去重
    for value in uniqueVals:
        subLabels=labels[:]
        myTree[bestFeatLabel][value]=createTree(splitDataSet(dataSet,bestFeat,value),subLabels) #递归调用
    return myTree

#### 6、使用树
inputTree -- 构建的决策树  
featLabels -- 特征标签列表  
testVec -- 册数数据

In [42]:
def classify(inputTree,featLabels,testVec):
    #firstStr=inputTree.keys()[0] # 获得第一个划分数据集的特征
    firstStr=list(inputTree.keys())[0]
    secondDict=inputTree[firstStr] #获得子集
    featIndex=featLabels.index(firstStr)
    for key in secondDict.keys():
        if testVec[featIndex]==key:
            if type(secondDict[key]).__name__=='dict': #若子集非叶子节点，则遍历
                classLabel=calssify(secondDict[key],featLabels,testVec)
            else: #叶子节点，返回结果
                classLabel=secondDict[key]
    return classLabel

注：py3中 firstStr=inputTree.keys()[0] 这句代码报错如下：  
'dict_keys' object does not support indexing  
在python2.x中，dict.keys()返回一个列表，在python3.x中，dict.keys()返回一个dict_keys对象，比起列表，这个对象的行为更像是set，所以不支持索引的。  
使用list(dict.keys())[index]  

In [33]:
def storeTree(inputTree,filename):
    import pickle
    fw=open(filename,'w')
    pickle.dump(inputTree,fw)
    fw.close()

In [34]:
def grabTree(filename):
    import pickle
    fr=open(filename)
    return pickle.load(fr)

In [41]:
fr=open('lenses.txt')
lenses=[inst.strip().split('\t') for inst in fr.readlines()]
print(lenses)
lensesLabels=['age','prescript','astigmatic','tearRate']
lensesTree=createTree(lenses,lensesLabels)
print(lensesTree)

[['young', 'myope', 'no', 'reduced', 'no lenses'], ['young', 'myope', 'no', 'normal', 'soft'], ['young', 'myope', 'yes', 'reduced', 'no lenses'], ['young', 'myope', 'yes', 'normal', 'hard'], ['young', 'hyper', 'no', 'reduced', 'no lenses'], ['young', 'hyper', 'no', 'normal', 'soft'], ['young', 'hyper', 'yes', 'reduced', 'no lenses'], ['young', 'hyper', 'yes', 'normal', 'hard'], ['pre', 'myope', 'no', 'reduced', 'no lenses'], ['pre', 'myope', 'no', 'normal', 'soft'], ['pre', 'myope', 'yes', 'reduced', 'no lenses'], ['pre', 'myope', 'yes', 'normal', 'hard'], ['pre', 'hyper', 'no', 'reduced', 'no lenses'], ['pre', 'hyper', 'no', 'normal', 'soft'], ['pre', 'hyper', 'yes', 'reduced', 'no lenses'], ['pre', 'hyper', 'yes', 'normal', 'no lenses'], ['presbyopic', 'myope', 'no', 'reduced', 'no lenses'], ['presbyopic', 'myope', 'no', 'normal', 'no lenses'], ['presbyopic', 'myope', 'yes', 'reduced', 'no lenses'], ['presbyopic', 'myope', 'yes', 'normal', 'hard'], ['presbyopic', 'hyper', 'no', 'redu

In [43]:
lensesLabels=['age','prescript','astigmatic','tearRate']
result=classify(lensesTree,lensesLabels,['young','myope','yes','reduced'])
print(result)

no lenses
